In [12]:
import bs4 
import re 
import os 
import json
import requests
import datetime
import urllib
from dateutil import parser
from tqdm.notebook import tqdm
import pandas as pd
pd.set_option('max_colwidth', 300)
pd.set_option('max_columns', 100)

## Fun Fact.. Dealnews uses an API 

In [13]:
headers = {'Authorization': 'DN jxqfz29pbv9xpWSYnmJX'}

In [40]:
dealnews = requests.get('https://api.dealnews.com/content?facet_ids=1780&count=70'
                        , headers=headers)
dealnews.status_code

200

In [41]:
txt = json.loads(dealnews.content)
len(str(txt))

689542

In [42]:
txt.keys()

dict_keys(['additional_content_ids', 'brands', 'categories', 'content', 'deal_types', 'facet_groups', 'not_returned_content_ids', 'overflow_content_ids', 'returned_content_ids', 'vendors'])

In [102]:
iter1 = iter(txt)

In [ ]:
temp = next(iter1)
print(f"\nName: {temp}")
txt[temp]

## Understanding API Options

In [142]:
## Deal Types
keeper_elems = ['name', 'count', 'deal_type_id', 'short_name']
groups = txt['deal_types']
key_group_info = []
for group in groups:
    key_group_info.append([group[x] for x in keeper_elems])
deal_types_df = pd.DataFrame(key_group_info, columns = keeper_elems)
deal_types_df['id_name'] = 'deal_type_id'
deal_types_df

,name,count,deal_type_id,short_name,id_name
0,Product Discounts,1470,1,Product Discounts,deal_type_id
1,Promo Codes,32,2,Promo Codes,deal_type_id
2,Store Sales & Events,680,3,Store Sales,deal_type_id


In [143]:
## Categories
keeper_elems = ['name', 'count', 'category_id', 'short_name']
groups = txt['categories']
key_group_info = []
for group in groups:
    key_group_info.append([group[x] for x in keeper_elems])
category_df = pd.DataFrame(key_group_info, columns = keeper_elems)
category_df['id_name'] = 'category_id'
category_df

,name,count,category_id,short_name,id_name
0,Home & Garden,836,196,Home & Garden,category_id
1,Clothing & Accessories,539,202,Clothing & Accessories,category_id
2,Electronics,201,142,Electronics,category_id
3,Computers,186,39,Computers,category_id
4,Store Events,136,40,Store Events,category_id
5,Sports & Fitness,117,211,Sports & Fitness,category_id
6,Health & Beauty,113,756,Health & Beauty,category_id
7,Gaming & Toys,96,186,Gaming & Toys,category_id
8,Office Supplies,43,182,Office Supplies,category_id
9,Automotive,41,238,Automotive,category_id


In [144]:
## Brands
keeper_elems = ['name', 'count', 'brand_id', 'short_name']
groups = txt['brands']
key_group_info = []
for group in groups:
    key_group_info.append([group[x] for x in keeper_elems])
brand_df = pd.DataFrame(key_group_info, columns = keeper_elems)
brand_df['id_name'] = 'brand_id'
brand_df

,name,count,brand_id,short_name,id_name
0,Apple,32,13,Apple,brand_id
1,Nike,26,828,Nike,brand_id
2,Dell,20,14,Dell,brand_id
3,DeWalt,19,313,DeWalt,brand_id
4,Lenovo,18,406,Lenovo,brand_id
...,...,...,...,...,...
517,Jacked Factory,1,65756,Jacked Factory,brand_id
518,Resideo,1,65759,Resideo,brand_id
519,Better Wood Products,1,65762,Better Wood Products,brand_id
520,WeePro,1,65765,WeePro,brand_id


In [145]:
## Vendors
keeper_elems = ['name', 'count', 'vendor_id', 'short_name']
groups = txt['vendors']
key_group_info = []
for group in groups:
    key_group_info.append([group[x] for x in keeper_elems])
vendor_df = pd.DataFrame(key_group_info, columns = keeper_elems)
vendor_df['id_name'] = 'vendor_id'
vendor_df

,name,count,vendor_id,short_name,id_name
0,Amazon,714,313,Amazon,vendor_id
1,eBay,128,50,eBay,vendor_id
2,Home Depot,109,958,Home Depot,vendor_id
3,Macy's,56,288,Macy's,vendor_id
4,Walmart,40,321,Walmart,vendor_id
...,...,...,...,...,...
381,MQ Direct,1,50263,MQ Direct,vendor_id
382,KiwiCo,1,50977,KiwiCo,vendor_id
383,SunPower Corporation,1,54593,SunPower Corporation,vendor_id
384,TruGreen,1,54596,TruGreen,vendor_id


In [146]:
## Facet Groups 
keeper_elems = ['name', 'count', 'facet_id', 'short_name']
facet_group = txt['facet_groups']
key_group_info = []
for facet in facet_group:
    groups = facet['facets']
    for group in groups:
        key_group_info.append([group[x] for x in keeper_elems])
facet_df = pd.DataFrame(key_group_info, columns = keeper_elems)
facet_df['id_name'] = 'facet_id'
facet_df

,name,count,facet_id,short_name,id_name
0,Popularity: 5/5,157,1774,Popularity: 5/5,facet_id
1,Popularity: 4/5,622,1777,Popularity: 4/5,facet_id
2,Popularity: 3/5,2182,1780,Popularity: 3/5,facet_id
3,Popularity: 2/5,1382,1783,Popularity: 2/5,facet_id
4,Popularity: 1/5,3831,1786,Popularity: 1/5,facet_id
5,Staff Pick,119,1682,Staff Pick,facet_id
6,Freebies,26,1742,Freebies,facet_id
7,Top Tech,114,1748,Top Tech,facet_id
8,Under $25,215,1751,Under $25,facet_id
9,Exclusive Offers,61,1792,Exclusive Offers,facet_id


### Grouping Options together

In [148]:
df_type = {"deal_type" : deal_types_df,
"vendor" : vendor_df,
"facet_group" : facet_df,
"brand" : brand_df,
"category" : category_df}


df_type = {k : v.rename(columns = lambda x: 'id' 
                        if re.search(re.compile('_id$'), x) != None 
                        else x)
           for k, v in df_type.items()}

all_dfs = pd.concat(df_type)
all_dfs.head()

name  count   id         short_name       id_name
deal_type 0     Product Discounts   1470    1  Product Discounts  deal_type_id
          1           Promo Codes     32    2        Promo Codes  deal_type_id
          2  Store Sales & Events    680    3        Store Sales  deal_type_id
vendor    0                Amazon    714  313             Amazon     vendor_id
          1                  eBay    128   50               eBay     vendor_id

In [159]:
def get_summary1(df):
    out = {}
    out['Offers Sum'] = df['count'].sum()
    out['Group Count'] = df.shape[0]
    df = df.sort_values(by='count', ascending=False)
    most_common = df.iloc[0]
    out['Most Common Name'] = most_common['name']
    out['Most Common Count'] = most_common['count']
    out['Most Common ID'] = most_common['id']
    
    out_series = pd.Series(out)
    return(out_series)

In [160]:
summary1 = all_dfs.reset_index().groupby('level_0')\
    .apply(get_summary1)
summary1

,Offers Sum,Group Count,Most Common Name,Most Common Count,Most Common ID
level_0,,,,,
brand,1044,522,Apple,32,13
category,2390,16,Home & Garden,836,196
deal_type,2182,3,Product Discounts,1470,1
facet_group,9857,32,Popularity: 1/5,3831,1786
vendor,2170,386,Amazon,714,313


In [53]:
facet_groups = [x['name'] for x in txt['facet_groups']]
facet_groups

['Popularity Rank', 'Collection', 'Condition', 'Offer Status', 'Events']

In [56]:
content1 = txt['content'][0]
list(content1.keys())

['type',
 'id',
 'headline',
 'brief_headline',
 'summary',
 'secondary_summary',
 'teaser',
 'body',
 'key_attribute',
 'brief_notes',
 'coupon_code',
 'url',
 'offsite_url',
 'show_code_url',
 'images',
 'display_date',
 'publish_datetime_ts',
 'update_datetime_ts',
 'expiration_datetime_ts',
 'last_verified_datetime_ts',
 'editors_choice',
 'sponsored',
 'flat_fee',
 'expired',
 'expires_today',
 'exclusive',
 'searchable',
 'hotness',
 'call_out',
 'call_out_comparison',
 'sub_call_out',
 'comment_count',
 'comments_url',
 'shareable',
 'category',
 'categories',
 'vendor',
 'sponsored_vendor',
 'product',
 'author',
 'facet_groups',
 'media_call_out',
 'views',
 'available_start_datetime_ts',
 'available_end_datetime_ts',
 'timestamp',
 'additional_information',
 'summary_required',
 'features',
 'deal_type',
 'value_statement']

## Parse One API Section Request

In [46]:

def parse_easy_elems(content):
    content_keys_to_keep = [
         'id',
         'headline',
         'brief_headline',
         'summary',
         'secondary_summary',
         'key_attribute',
         'brief_notes',
         'url',
         'display_date',
         'publish_datetime_ts',
         'update_datetime_ts',
         'expiration_datetime_ts',
         'last_verified_datetime_ts',
         'editors_choice',
         'sponsored',
         'expired',
         'expires_today',
         'exclusive',
         'searchable',
         'hotness',
         'call_out',
         'call_out_comparison',
         'sub_call_out'
    ]
    out_dict = {x : content[x] for x in content_keys_to_keep}
    return(out_dict)

def parse_time_fields(content):
    elements = ['publish_datetime_ts',
         'update_datetime_ts',
         'expiration_datetime_ts',
         'last_verified_datetime_ts']
    out_dict = {}
    for x in elements:
        temp_time = content.get(x) if content.get(x) != None else 0
        out_dict[x] = datetime.datetime.fromtimestamp(temp_time)
    return(out_dict)

def parse_coupon_code(content):
    cc_list = content['coupon_code']
    cc_code = cc_list[0] if len(cc_list) > 0 else None 
    out_dict = {"Coupon Code":cc_code}
    return(out_dict)

def _parse_category(cat_dict):
    keeper_elements = ['category_id', 'name'
                      'path', 'ancestor_list' ]
    cat_elems = {x:cat_dict.get(x) for x in keeper_elements}
    return(cat_elems)

def parse_2_categories(content):
    cat_list = content['categories']
    out_dict = {}
    # Only keep 2 categories 
    if len(cat_list) < 2:
        cat_list.append({})
    for dict_num in range(2):
        temp_cat_elems = _parse_category(cat_list[dict_num])
        temp_cat_elems = {key + '_' +str(dict_num): value 
                          for key, value in temp_cat_elems.items()}
        out_dict.update(temp_cat_elems)
    
    return(out_dict)

def parse_vendor(content):
    vend_dict = content['vendor']
    keeper_elems = ['vendor_id', 'name']
    vend_dict = {x:vend_dict[x] for x in keeper_elems}
    return(vend_dict)

def _create_img_path(content):
    # Create Filename
    headline = content.get('headline')
    id1 = content.get('id')
    file_name = headline + '_' + str(id1) + '.jpg'
    file_name = file_name.replace('/', '')
    # Create Directory structure 
    date = str(datetime.datetime.now().date())
    if 'base_path' not in locals():
        base_path = os.getcwd() + '/Dealnews Images/'
    path = base_path + date + '/'
    os.makedirs(path, exist_ok=True)
    #File Save location 
    save_location = path + file_name
    return(save_location)
    
def parse_image(content):
    image_link = content.get('images').get('XXL').get('url')
    save_path = _create_img_path(content)
    
    urllib.request.urlretrieve(image_link, save_path)
    out_dict = {'Image path': save_path}
    return(out_dict)


In [47]:
def parse_dn_item(content):
    all_details = {}
    
    main_elems = parse_easy_elems(content)
    all_details.update(main_elems)
    
    time_fields = parse_time_fields(content)
    all_details.update(time_fields)
    
    cc_code = parse_coupon_code(content)
    all_details.update(cc_code)
    
    categories = parse_2_categories(content)
    all_details.update(categories)

    vendor = parse_vendor(content)
    all_details.update(vendor)
    
    image_info = parse_image(content)
    all_details.update(image_info)
    
    return(all_details)

In [48]:
elems = parse_dn_item(content1)
elems_series = pd.Series(elems)


In [49]:
page_items = []
for content1 in tqdm(txt['content']):
    elems = parse_dn_item(content1)
    elems_series = pd.Series(elems)
    page_items.append(elems_series)

In [50]:
dn_today = pd.concat(page_items, axis =1).T
dn_today

,id,headline,brief_headline,summary,secondary_summary,key_attribute,brief_notes,url,display_date,publish_datetime_ts,update_datetime_ts,expiration_datetime_ts,last_verified_datetime_ts,editors_choice,sponsored,expired,expires_today,exclusive,searchable,hotness,call_out,call_out_comparison,sub_call_out,Coupon Code,category_id_0,namepath_0,ancestor_list_0,category_id_1,namepath_1,ancestor_list_1,vendor_id,name,Image path
0,19240289,Uxu Edge Protector,Uxu Edge Protector for $3,"Apply coupon code ""E5FSWM8M"" for a savings of $14.","<div class=""body-offer body-sustain""> <div class=""snippet summary"" title=""Apply&#x20;code&#x20;&quot;E5FSWM8M&quot;&#x20;to&#x20;save&#x20;&#x24;14.""> <p>Apply code ""E5FSWM8M"" to save $14. <a target=""_blank"" href=""https://www.dealnews.com/lw/click.html?20,2,19240289,iref=site-widget"">Buy Now at ...",Amazon,"Free shipping w/ prime. Sold by Uxu Shop via Amazon. Available in several colors (Coffee pictured). <br><b>Features:</b> made of NBR foam, toxin-, odor-, SCCP-, and phthalate-free and 18-feet edge and 4 corner pieces. Model: UX-BSBREP05-Cal01.",https://www.dealnews.com/products/Private-Label-Brands/Uxu-Edge-Protector/221246.html?iref=site-widget,Last verified 1 hr 16 min ago,2021-05-17 19:20:09,2021-05-28 21:54:26,2021-05-31 06:59:00,2021-05-28 20:57:38,False,False,False,False,False,True,3,$3.39,$17,free shipping w/ Prime,E5FSWM8M,196,None,None,None,None,None,313,Amazon,/home/malcolm/Demo with APIs/Dealnews Images/2021-05-28/Uxu Edge Protector_19240289.jpg
1,19241297,Naiyo Electric Bug Zapper,Naiyo Electric Bug Zapper for $17,"Apply coupon code ""S7G8BXWK"" for a savings of $17.","<div class=""body-offer body-sustain""> <div class=""snippet summary"" title=""Apply&#x20;coupon&#x20;code&#x20;&quot;S7G8BXWK&quot;&#x20;for&#x20;a&#x20;savings&#x20;of&#x20;&#x24;17.""> <p>Apply coupon code ""S7G8BXWK"" for a savings of $17. <a target=""_blank"" href=""https://www.dealnews.com/lw/click.h...",Amazon,"Free shipping. Sold by Guanfoo via Amazon. <br><b>Features:</b> 4,200V, 60Hz frequency, removable tray and waterproof design.",https://www.dealnews.com/Naiyo-Electric-Bug-Zapper-for-17-free-shipping/19241297.html?iref=site-widget,Last verified 1 hr 15 min ago,2021-05-18 17:29:25,2021-05-28 21:44:23,2021-06-05 06:59:00,2021-05-28 20:58:05,False,False,False,False,False,True,3,$17,$34,free shipping,S7G8BXWK,198,None,196,None,None,None,313,Amazon,/home/malcolm/Demo with APIs/Dealnews Images/2021-05-28/Naiyo Electric Bug Zapper_19241297.jpg
2,19243709,eBay Camping Sale,eBay Camping Sale: Up to 55% off,"Save on coolers, tents, tables, chairs, torches, and more.","<div class=""snippet summary"" title=""Save&#x20;on&#x20;coolers,&#x20;tents,&#x20;tables,&#x20;chairs,&#x20;torches,&#x20;and&#x20;more.""> <p>Save on coolers, tents, tables, chairs, torches, and more. <a target=""_blank"" href=""https://www.dealnews.com/lw/click.html?20,2,19243709,iref=site-widget"">S...",eBay,Free shipping. Pictured is the Tahoe Gear Olympia 10-Person 3-Season Camping Tent for $209.99 ($150 off),https://www.dealnews.com/eBay-Camping-Sale-Up-to-55-off-free-shipping/19243709.html?iref=site-widget,Last verified 4 hr 11 min ago,2021-05-28 08:21:28,2021-05-28 21:03:54,1970-01-01 00:00:00,2021-05-28 18:02:24,False,False,False,False,False,True,3,up to 55% off,,free shipping,None,298,None,211,None,None,None,50,eBay,/home/malcolm/Demo with APIs/Dealnews Images/2021-05-28/eBay Camping Sale_19243709.jpg
3,19252157,Huanuo Dual Monitor and Laptop Mount Stand,Huanuo Dual Monitor and Laptop Mount Stand for $40,It's $60 under list price.,"<div class=""body-offer body-sustain""> <div class=""snippet summary"" title=""Clip&#x20;the&#x20;60&#x25;&#x20;extra&#x20;savings&#x20;coupon&#x20;to&#x20;get&#x20;this&#x20;price.""> <p>Clip the 60% extra savings coupon to get this price. <a target=""_blank"" href=""https://www.dealnews.com/lw/click.ht...",Amazon,"Free shipping. Sold by Huanuo Shop via Amazon. <br><b>Features:</b> fits 13"" to 24"" monit

In [51]:
dn_today[['headline', 'brief_headline', 'summary', 'key_attribute'
          , 'call_out', 'call_out_comparison']]

,headline,brief_headline,summary,key_attribute,call_out,call_out_comparison
0,Uxu Edge Protector,Uxu Edge Protector for $3,"Apply coupon code ""E5FSWM8M"" for a savings of $14.",Amazon,$3.39,$17
1,Naiyo Electric Bug Zapper,Naiyo Electric Bug Zapper for $17,"Apply coupon code ""S7G8BXWK"" for a savings of $17.",Amazon,$17,$34
2,eBay Camping Sale,eBay Camping Sale: Up to 55% off,"Save on coolers, tents, tables, chairs, torches, and more.",eBay,up to 55% off,
3,Huanuo Dual Monitor and Laptop Mount Stand,Huanuo Dual Monitor and Laptop Mount Stand for $40,It's $60 under list price.,Amazon,$40,$100
4,Homegeek 1.2-Liter 300W Mini Food Processor,Homegeek 1.2-Liter 300W Mini Food Processor for $18,"Apply coupon code ""H33668USZW"" for a savings of $12.",Amazon,$18,$30
...,...,...,...,...,...,...
65,Patagonia Sale at Paragon Sports,Patagonia Sale at Paragon Sports: Up to 30% off,"Save on a small, eclectic selection of mostly women's jackets, swimwear, and more.",Paragon Sports,up to 30% off,
66,"Dell Inspiron 5000 11th-Gen. i3 2-in-1 14"" Touch Laptop","Dell Inspiron 5000 11th-Gen. i3 2-in-1 14"" Touch Laptop for $350","It's $30 cheaper than it was three days ago, when it was already discounted significantly.",Dell Home,$350,
67,Callaway Golf Men's Faux Down Puffer Jacket,Callaway Golf Men's Faux Down Puffer Jacket for $22,It's $68 under list price.,Nordstrom Rack,$22,$90
68,Father's Day T-Shirts at Tanga,Father's Day T-Shirts at Tanga: from $14,"Apply coupon code ""DEALNEWS"" to save.",Tanga,from $14,


In [52]:
dn_today['key_attribute'].value_counts()

Amazon                       24
eBay                          5
Walmart                       3
Nordstrom Rack                3
Kohl's                        2
Banana Republic Factory       2
Home Depot                    2
Dell Home                     2
Harman Audio                  1
Academy Sports & Outdoors     1
Aukey                         1
Tanga                         1
Sears                         1
Men's Wearhouse               1
That Daily Deal               1
Staples                       1
Flash PopUp                   1
Best Buy                      1
Paragon Sports                1
Northern Tool                 1
Petco                         1
Target                        1
WatchMaxx                     1
RVShare                       1
Adorama                       1
Clarks                        1
Nautica                       1
Dell Technologies             1
Speedo                        1
De Novo                       1
Shoebacca                     1
Jos. A. 

In [ ]:
time1 = dn_today['publish_datetime_ts'].iloc[0]
time1

In [ ]:
 datetime.datetime.fromtimestamp(time1)

In [ ]:
print(dn_today.columns)

In [ ]:
dn_today.head()

## Also the RSS Feed

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
link = 'https://www.dealnews.com/rss/todays-edition'

In [ ]:
page = requests.get(link)
page.status_code

In [ ]:
rss_feed = BeautifulSoup(page.content, features='xml')
rss_feed

In [ ]:
items = rss_feed.find_all('item')
len(items)

In [ ]:
temp = items[0]
temp

In [ ]:
temp.find('title').text

In [ ]:
def parse_rss_item(rss_item):
    keeper_elems = ['title', 'link', 'description', 'pubDate']
    out_dict = {x : rss_item.find(x).text for x in keeper_elems}
    out = pd.Series(out_dict)
    return(out)

In [ ]:
all_items = [parse_rss_item(x) for x in items]

In [ ]:
all_items_df = pd.concat(all_items, axis =1).T
all_items_df.head()

## Third way would be to build request site and scrape from there   

